<a href="https://colab.research.google.com/github/lenust/leaf_damage_neuronet/blob/main/UnetPlusPlus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Загрузка модели:

In [ ]:
import segmentation_models_pytorch as smp
import torch
model = smp.UnetPlusPlus(encoder_name="efficientnet-b0", classes = 1, activation = "sigmoid")
model.load_state_dict(torch.load("/content/drive/MyDrive/нейросеть/leaves/SMP_dataset2_UnetPlusPlus_FocalLoss.pth"))
model.to(device)

In [ ]:
import os
import torch
from PIL import Image
import numpy as np
import cv2
import torchvision.transforms as transforms
import torchvision.datasets as datasets

def load_image(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return transforms.Compose([ transforms.ToTensor(), transforms.Resize((224, 224))])(img)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, folder1):
        self.folder1 = folder1
        self.filenames = [f for f in os.listdir(folder1)]

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, index):
        filename = self.filenames[index]
        img1_path = os.path.join(self.folder1, filename)
        img1 = load_image(img1_path)
        return img1

true_test = CustomDataset('/content/drive/MyDrive/нейросеть/leaves/true_test') #указать папку с исходными изображениями


def show_valset_pred(model, cols=16):
    images, pred_masks = [], []
    for batch in true_test:
        with torch.no_grad():
            img = batch
            images.append(binarize_image(img, 0.6))
            output = model(img.unsqueeze(0).to(device))
            pred_masks.append(output.cpu())
    show(torch.stack(images)[:cols, ...])
    show(torch.stack(pred_masks).squeeze(1)[:cols, ...])
    show(torch.stack(pred_masks).squeeze(1)[:cols, ...] - torch.stack(images)[:cols, ...])

show_valset_pred(model)

Вычисление площади:

In [ ]:
import os
from PIL import Image

folder = '/content/drive/MyDrive/нейросеть/leaves/true_test' #указать папку с исходными изображениями
image_sizes = []

for filename in os.listdir(folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        filepath = os.path.join(folder, filename)
        with Image.open(filepath) as img:
            image_sizes.append(img.size)

image_sizes = [(y, x) for x, y in image_sizes]

def resize_images(images, sizes):
    resized_images = []
    for img, size in zip(images, sizes):
        resized_img = torch.nn.functional.interpolate(img, size=size, mode='bilinear', align_corners=False)
        resized_images.append(resized_img)
    return resized_images

images, pred_masks = [], []
for batch in true_test:
    with torch.no_grad():
        img = batch
        images.append(binarize_image(img, 0.6).unsqueeze(0))
        output = model(img.unsqueeze(0).to(device))
        pred_masks.append(output.cpu())

resized_images = resize_images(images, image_sizes)
resized_masks = resize_images(pred_masks, image_sizes)

def subtract_tensor_lists(list1, list2):
    result = []
    for t1, t2 in zip(list1, list2):
        subtracted_tensor = binarize_image(t1 - t2, 0.05)
        result.append(subtracted_tensor.squeeze())
    return result

# Выполняем операцию вычитания одного списка из другого
subtracted_list = subtract_tensor_lists(resized_masks, resized_images)

import matplotlib.pyplot as plt

plt.imshow(subtracted_list[5], cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
import numpy as np

def count_black_pixels(image_tensor):
    # Преобразуем тензор в массив NumPy
    image_array = np.array(image_tensor)

    # Подсчитываем количество черных пикселей
    num_black_pixels = np.sum(image_array == 0)

    return num_black_pixels

square  = []
for tensor in subtracted_list:
  square.append(count_black_pixels(tensor))

print(square)